In [ ]:
class Symbol(object):
    def __init__(self, value):
        self.value = value


if __name__ == "__main__":
    x = Symbol("Py")
    y = Symbol("Py")

    symbols = set()
    symbols.add(x)
    symbols.add(y)

    print(x is y)
    print(x == y)
    print(len(symbols))

    # 두 변수 x y의 참조가 달라서 false가 나옴


False
False
2


In [ ]:
class Symbol(object):
    def __init__(self, value):
        self.value = value

    def __eq__(self, other):
        if isinstance(self, other.__class__):
            return self.value == other.value
        else:
            return NotImplemented


if __name__ == "__main__":
    x = Symbol("Py")
    y = Symbol("Py")

    symbols = set()
    symbols.add(x)
    symbols.add(y)

    print(x is y)
    print(x == y)
    print(len(symbols))

     # 객체가 해시 가능하지 않다 = 가변객체

TypeError: ignored

In [ ]:
class Symbol(object):
    def __init__(self, value):
        self.value = value

    def __eq__(self, other):
        if isinstance(self, other.__class__):
            return self.value == other.value
        else:
            return NotImplemented

    def __hash__(self):
        return hash(self.value)


if __name__ == "__main__":
    x = Symbol("Py")
    y = Symbol("Py")

    symbols = set()
    symbols.add(x)
    symbols.add(y)

    print(x is y)
    print(x == y)
    print(len(symbols))

    # 위의 에러를 고치기 위해 해시 메서드 입력

False
True
1


In [ ]:
import queue
import threading

q = queue.Queue()


def worker(num):
    while True:
        item = q.get()
        if item is None:
            break
        # 작업을 처리한다.
        print("스레드 {0} : 처리 완료 {1}".format(num+1, item))
        q.task_done()


if __name__ == "__main__":
    num_worker_threads = 5
    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker, args=(i,))
        t.start()
        threads.append(t)

    for item in range(20):
        q.put(item)

    # 모든 작업이 끝날 때까지 대기한다(block).
    q.join()

    # 워커 스레드를 종료한다(stop).
    for i in range(num_worker_threads):
        q.put(None)
    for t in threads:
        t.join()

스레드 2 : 처리 완료 0
스레드 5 : 처리 완료 1
스레드 3 : 처리 완료 2스레드 2 : 처리 완료 3
스레드 2 : 처리 완료 4
스레드 2 : 처리 완료 5
스레드 2 : 처리 완료 6
스레드 2 : 처리 완료 7
스레드 2 : 처리 완료 8
스레드 2 : 처리 완료 9
스레드 2 : 처리 완료 10
스레드 1 : 처리 완료 11
스레드 4 : 처리 완료 12
스레드 4 : 처리 완료 13
스레드 4 : 처리 완료 14
스레드 4 : 처리 완료 15
스레드 4 : 처리 완료 16
스레드 4 : 처리 완료 17
스레드 4 : 처리 완료 18
스레드 4 : 처리 완료 19



In [ ]:
# 뮤텍스는 1부터 시작하는 정수
from threading import Thread, Lock
import threading


def worker(mutex, data, thread_safe):
    if thread_safe:
        mutex.acquire()
    try:
        print("스레드 {0}: {1}\n".format(threading.get_ident(), data))
    finally:
        if thread_safe:
            mutex.release()


if __name__ == "__main__":
    threads = []
    thread_safe = True
    mutex = Lock()
    for i in range(20):
        t = Thread(target=worker, args=(mutex, i, thread_safe))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()

스레드 139752520263424: 0

스레드 139752553834240: 1

스레드 139752553834240: 2

스레드 139752553834240: 3

스레드 139752520263424: 4

스레드 139752553834240: 5

스레드 139752553834240: 6

스레드 139752520263424: 7

스레드 139752553834240: 8

스레드 139752520263424: 9

스레드 139752553834240: 10

스레드 139752520263424: 11

스레드 139752553834240: 12

스레드 139752553834240: 13

스레드 139752520263424: 14

스레드 139752553834240: 15

스레드 139752520263424: 16

스레드 139752537048832: 17

스레드 139752537048832: 18

스레드 139752537048832: 19



In [ ]:
# 세마포어는 1보다 큰 수로 시작할 수 있다
# 뮤텍스의 락 및 언락 작업과 유사한 작업을 지원한다
import threading
import time


class ThreadPool(object):
    def __init__(self):
        self.active = []
        self.lock = threading.Lock()

    def acquire(self, name):
        with self.lock:
            self.active.append(name)
            print("획득: {0} | 스레드 풀: {1}".format(name, self.active))

    def release(self, name):
        with self.lock:
            self.active.remove(name)
            print("반환: {0} | 스레드 풀: {1}".format(name, self.active))


def worker(semaphore, pool):
    with semaphore:
        name = threading.currentThread().getName()
        pool.acquire(name)
        time.sleep(1)
        pool.release(name)


if __name__ == '__main__':
    threads = []
    pool = ThreadPool()
    semaphore = threading.Semaphore(3)
    for i in range(10):
        t = threading.Thread(
            target=worker, name="스레드 " + str(i), args=(semaphore, pool))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()


획득: 스레드 0 | 스레드 풀: ['스레드 0']
획득: 스레드 1 | 스레드 풀: ['스레드 0', '스레드 1']
획득: 스레드 2 | 스레드 풀: ['스레드 0', '스레드 1', '스레드 2']
반환: 스레드 0 | 스레드 풀: ['스레드 1', '스레드 2']
반환: 스레드 1 | 스레드 풀: ['스레드 2']
반환: 스레드 2 | 스레드 풀: []
획득: 스레드 4 | 스레드 풀: ['스레드 4']
획득: 스레드 5 | 스레드 풀: ['스레드 4', '스레드 5']
획득: 스레드 3 | 스레드 풀: ['스레드 4', '스레드 5', '스레드 3']
반환: 스레드 4 | 스레드 풀: ['스레드 5', '스레드 3']
반환: 스레드 5 | 스레드 풀: ['스레드 3']
획득: 스레드 7 | 스레드 풀: ['스레드 3', '스레드 7']
획득: 스레드 6 | 스레드 풀: ['스레드 3', '스레드 7', '스레드 6']
반환: 스레드 3 | 스레드 풀: ['스레드 7', '스레드 6']
획득: 스레드 8 | 스레드 풀: ['스레드 7', '스레드 6', '스레드 8']
반환: 스레드 7 | 스레드 풀: ['스레드 6', '스레드 8']
반환: 스레드 6 | 스레드 풀: ['스레드 8']
획득: 스레드 9 | 스레드 풀: ['스레드 8', '스레드 9']
반환: 스레드 8 | 스레드 풀: ['스레드 9']
반환: 스레드 9 | 스레드 풀: []
